In [98]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from text_processing import preprocess_text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_excel('../data.xlsx', nrows=7600)

In [99]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Tokenize and preprocess the text
texts = df['text'].astype(str).apply(preprocess_text).values

In [100]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences)
print(data.shape)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


(7590, 28)


In [101]:
labels = df['id'].values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

with open('encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [102]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42, shuffle=True)

In [103]:
embedding_dim = 90
LSTM1 = 80
LSTM2 = 90
optimalizer = 'adam'
batch_size = 2
epochs = 8

In [104]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBEDDING_DIM = embedding_dim
MAX_LEN = len(data[0])
print(MAX_LEN)
OUTPUT_CLASSES = len(y_train[0])
print(OUTPUT_CLASSES)

28
33


Model

In [107]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(LSTM(LSTM1, return_sequences=True))  # Změna na True pro více vrstev LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(LSTM(LSTM2))  # Další vrstva LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(Dense(OUTPUT_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimalizer,
              metrics=['accuracy'])


In [108]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,  # Zvýšení počtu epoch
          validation_data=(X_test, y_test))

Epoch 1/8
2657/2657 [==============================] - 50s 18ms/step - loss: 1.9888 - accuracy: 0.4321 - val_loss: 0.7031 - val_accuracy: 0.8085
Epoch 2/8
2657/2657 [==============================] - 53s 20ms/step - loss: 0.5289 - accuracy: 0.8460 - val_loss: 0.4626 - val_accuracy: 0.8718
Epoch 3/8
2657/2657 [==============================] - 54s 20ms/step - loss: 0.2784 - accuracy: 0.9162 - val_loss: 0.3773 - val_accuracy: 0.8906
Epoch 4/8
2657/2657 [==============================] - 55s 21ms/step - loss: 0.1635 - accuracy: 0.9484 - val_loss: 0.3690 - val_accuracy: 0.9029
Epoch 5/8
2657/2657 [==============================] - 56s 21ms/step - loss: 0.1369 - accuracy: 0.9554 - val_loss: 0.3597 - val_accuracy: 0.9082
Epoch 6/8
2657/2657 [==============================] - 58s 22ms/step - loss: 0.1038 - accuracy: 0.9665 - val_loss: 0.3644 - val_accuracy: 0.9130
Epoch 7/8
2657/2657 [==============================] - 58s 22ms/step - loss: 0.0802 - accuracy: 0.9721 - val_loss: 0.3669 - val_ac

In [131]:
model.save('text_model_v2')

INFO:tensorflow:Assets written to: text_model_v2\assets


INFO:tensorflow:Assets written to: text_model_v2\assets


In [133]:
from keras import models
from keras.models import load_model
import pickle
from text_processing import preprocess_text
model = load_model('../text_model_v2')

In [161]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from text_processing import preprocess_text
class_names = ["lukostřelba", "baseball", "basketbal", "kulečník", "bmx", "bowling", "box", "jízda na býku", "roztleskávání", "curling", "šerm", "krasobruslení", "fotbal", "závody formule 1", "golf", "skok do výšky", "hokej", "dostihy", "hydroplánové závody", "judo", "motocyklové závody", "pole dance", "rugby", "skoky na lyžích", "snowboarding", "rychlobruslení", "surfování", "plavání", "stolní tenis", "tenis", "dráhové kolo", "volejbal", "vzpírání"]
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)
    string = preprocess_text('vodní sport, kde se plavci snaží dostat co nejrychleji do cíle')
sequences = tokenizer.texts_to_sequences([string])
data = pad_sequences(sequences, maxlen=28)
predictions = model.predict(data)
predicted_labels = predictions.argmax(axis=-1)
predicted_labels = encoder.inverse_transform(predicted_labels)
predicted_confidence = predictions.max(axis=-1)
print(class_names[predicted_labels[0]])
print(f"jistota: {predicted_confidence[0]*100} %")

1/1 [==============================] - 0s 64ms/step
plavání
jistota: 99.91538524627686 %
